In [1]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import Dict, designs, MetalGUI

In [2]:
from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
import numpy as np
from collections import OrderedDict
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital


In [3]:
design = designs.DesignPlanar(overwrite_enabled=True)

In [4]:
gui = MetalGUI(design)

In [5]:
design.variables['cpw_width']='20um'
design.variables['cpw_gap']='12um'

# 3 ressonadores com acoplamento capacitivo

In [44]:
cap_opt= Dict(
    pos_x='-4mm',
    pos_y='0mm',
    orientation='330',
    north_width='20um',
    north_gap='12um',
    south_width='20um',
    south_gap='12um',
    cap_width='5um',
    cap_gap='6um',
    cap_gap_ground='12um',
    finger_length='55um',
    finger_count='3',
    cap_distance='1um'
)
cap1 = CapNInterdigital(design, 'CAP1', options = cap_opt)

cap_opt.pos_x = '4mm'
cap_opt.orientation = '390'
cap2 = CapNInterdigital(design, 'CAP2', options=cap_opt)

cap_opt.pos_x = '0mm'
cap_opt.pos_y = '-7mm'
cap_opt.orientation='90'
cap3 = CapNInterdigital(design, 'CAP3', options=cap_opt)

res1 = RouteStraight(design, 'RES1', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'CAP1',
            pin = 'north_end'
        ),
        end_pin = Dict(
            component = 'CAP2',
            pin = 'north_end'
        )
    ),
    lead = Dict(start_straight = '500um', end_straight = '500um'),
    fillet='200um',
    hfss_wire_bonds = True
))

res2 = RouteStraight(design, 'RES2', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'CAP1',
            pin = 'south_end'
        ),
        end_pin = Dict(
            component = 'CAP3',
            pin = 'north_end'
        )
    ),
    lead = Dict(start_straight = '470um', end_straight = '470um'),
    fillet='200um',
    hfss_wire_bonds = True
))

res3 = RouteStraight(design, 'RES3', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'CAP2',
            pin = 'south_end'
        ),
        end_pin = Dict(
            component = 'CAP3',
            pin = 'south_end'
        )
    ),
    lead = Dict(start_straight = '510um', end_straight = '510um'),
    fillet='200um',
    hfss_wire_bonds = True
))

pad1 = LaunchpadWirebond(design, 'PAD1', options=Dict(
    pos_x='-6mm',
    pos_y='2mm',
    orientation='0',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

pad2 = LaunchpadWirebond(design, 'PAD2', options=Dict(
    pos_x='6mm',
    pos_y='2mm',
    orientation='180',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

from collections import OrderedDict
anchors = OrderedDict()
anchors[0] = np.array([-0.425,1])
anchors[1] = np.array([0,0.500])
anchors[2] = np.array([0.425,1])

res = RouteAnchors(design, 'RES', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'PAD1',
            pin = 'tie'
        ),
        end_pin = Dict(
            component = 'PAD2',
            pin = 'tie'
        )
    ),
    anchors = anchors,
    lead=Dict(start_straight='1mm', end_straight='1mm'),
    total_length='8.5mm',
    fillet='200um',
    hfss_wire_bonds = True
))

gui.rebuild()
gui.autoscale()

# 3 ressonadores com acoplamento indutivo

In [ ]:
design.delete_all_components()

In [ ]:
otg1 = OpenToGround(design, 'OTG1', options=Dict(
    pos_x='-2.5mm',
    pos_y='0',
    orientation='180',
    width='20um',
    gap='12um',
    termination_gap='12um'
))